In [26]:
from gqimax.instructor import Instructor
%load_ext autoreload
%autoreload 2
ins = Instructor(4)
for i in range(10000):
	ins.append("h", 0)
	ins.append("rx", 1, 0.78)
	ins.append("h", 2)
	ins.append("h", 0)
	ins.append("cx", [0, 1])
	ins.append("h", 2)
	ins.append("h", 2)
	ins.append("ry", 0, 0.56)
	ins.append("cx", [1, 2])
	ins.append("h", 1)
	ins.append("h", 3)
	ins.append("h", 3)
	ins.append("h", 3)
	ins.append("h", 3)
	ins.append("h", 0)
	ins.append("h", 1)
	ins.append("h", 2)
	ins.append("h", 3)
	ins.append("h", 0)
	ins.append("h", 2)
	ins.append("cx", [1, 3])
ins.operatoring()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
from gqimax.instructor import group_instructorss_by_qubits
print(len(ins.operators)*len(ins.operators[0]))
grouped_instructorss = group_instructorss_by_qubits(ins.operators, ins.num_qubits)


110011


In [11]:
from qimax.instructor import construct_lut_noncx
LUT = construct_lut_noncx(grouped_instructorss, ins.num_qubits)


In [12]:
from gqimax.instructor import construct_lut_noncx
LUT2 = construct_lut_noncx(grouped_instructorss, ins.num_qubits)
import numpy as np
np.allclose(LUT, LUT2.get())

True

In [44]:
from gqimax.instructor import group_instructorss_by_qubits
from gqimax.pstabilizer import PStabilizer
%load_ext autoreload
%autoreload 2
num_qubits = 3
num_layers = 2

ins = Instructor(num_qubits)
for k in range(num_layers):
	for i in range(num_qubits - 1):
		ins.append('cx', [i, i + 1])
	ins.append('cx', [num_qubits - 1, 0])
	for i in range(num_qubits):
		ins.append('rx', i, np.random.rand())
		ins.append('ry', i, np.random.rand())
		ins.append('rz', i, np.random.rand())
ins.operatoring()

grouped_instructorss = group_instructorss_by_qubits(ins.operators, ins.num_qubits)
ins.LUT = construct_lut_noncx(grouped_instructorss, ins.num_qubits)

stb = PStabilizer(0, num_qubits)
stb.map(ins)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


TypeError: list indices must be integers or slices, not ndarray

In [41]:
import qiskit

qc = qiskit.QuantumCircuit(num_qubits)
for k in range(num_layers):
	for i in range(num_qubits - 1):
		qc.cx(i, i + 1)
	qc.cx(num_qubits - 1, 0)
	for i in range(num_qubits):
		qc.rx(np.random.rand(), i)
		qc.ry(np.random.rand(), i)
		qc.rz(np.random.rand(), i)
	qc.rz(np.random.rand(), i)
